In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)

In [19]:
# use large scale here to get the output really close to the value in dataset, which is important to Median.
# Be careful, since we may need to calculate average of 2 values in case of no middle value in dataset,
# this means larger scale can still blow up circuit size, unlike Mode func that scale doesnt affect circuit size much.
scales = [7]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [20]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.median(x)

# error here in Median only matters in determining the median value in case it doesnt exist in dataset. (Avg of 2 middle values)
error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [21]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 7, param_scale: 7, scale_input_multiplier: 10) ------------->

+--------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error   | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+--------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.0015621185 | 0.003124237  | 0.003124237 | 0         | 0.0015621185   | 0.003124237      | 0.003124237   | 0             | 0.0000048804286    | 0.000031526106     | 0.000031526106         |
+--------------+--------------+-------------+-----------+----------------+------------------+--------------

scale:  [7]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[-24804,12494],"logrows":16,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":14432,"total_assignments":15668,"total_const_size":2421,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[14432,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"ReLU",{"Floor":{"scale":2.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709646739459}


In [22]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 6.426691770553589 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.546875
==== Generating Proof ====
proof:  {'instances': [['13613be83876b1d6edaa4022f0037a4a0dd2bca4e9a736c5ced55f1d1ec6fb29', '0100000000000000000000000000000000000000000000000000000000000000', 'c618000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0dbc445e36929eef3f692dfbba12b4651821021a036c9da26e820dfc7b4f82db29c599853933a6f7221d50335be0438f7c9623a6ac4124fa71cb70c13575024c1237cb8dc561707eb2fec8f268e933f403dd6b959ffe1af09e79ac53e6fb75f821e6a780ec7311f59ca9a67587f6f840113c2abca554d3207d4841769baf6c4727ad548c26deaed4d2efc1c7b8f62565c87765cfa421df584354e4b5930d9513166ce70f8c33e12b217428445d5a59f2535f7ae59f94f3d4ce22598ca615b14c0f8f72b6eb18c298cf08fc70af414f77e3ad296703128b7a86463a2df445027227796ce55b7e70f729bd36c614b16ebac8769a5e003087e42499cfe7da3d3b220760c38319bcb3580ce34bc4b36b713ccb325d817adb3c2b3bc02d5dba109a7b25d2dddbafd

In [24]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [49.546875]
